<a href="https://colab.research.google.com/github/stinh/Tiba_02_Crawler-and-SQL/blob/main/00_practice_20211013_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Part 1：抓經濟日報即時新聞
import requests
from bs4 import BeautifulSoup
import prettytable as pt

url = "https://money.udn.com/rank/newest/1001/0/1"
head = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
}

resp = requests.get(url, headers=head)

# 確認resp有拿到東西，就可以把resp.text丟到BeautifulSoup裡面去解析
source = BeautifulSoup(resp.text, "html.parser")
news_rows = source.find_all("tr", {"style": "table-row"})

# 確認底下forloop裡面資料都可以正常抓到以後，來建立一個prettytable存放資料
table = pt.PrettyTable(["標題", "類別", "時間", "網址"])
# 微調一下對齊方式
table.align["標題"] = "l"

# 剛剛抓到的news_rows是一個list，用for loop針對內容做進一步搜尋
for news in news_rows:
    # 抓出新聞的標題文字，找a標籤
    title = news.find("a").text
    # 抓出新聞類別所在的td標籤，需要詳細的屬性(attrs)
    category = news.find("td", {"class": "only_web"}).text
    # 抓出新聞發布時間
    time = news.find("td", {"align": "right", "class": "only_1280"}).text
    # 抓出新聞網址連結
    link = news.find("a").attrs["href"]
    # 把資料放進做好的table中
    table.add_row([title, category, time, link])

# 最後印出整個做好的table
print(table)


# Part 2: 將以上資料點進每個新聞連接後抓取新聞內文，並存檔出來
import codecs

# 這邊我想稍微處理一下存檔檔名當作每篇文章的標題，因為時間有標點符號，會用replace()方法稍做處理
for i, news in enumerate(news_rows):
    # 抓出新聞的標題文字，找a標籤
    title = news.find("a").text
    # 抓出新聞網址連結
    link = news.find("a").attrs["href"]
    # 抓出時間
    time = news.find("td", {"align": "right", "class": "only_1280"}).text
    time_mdf = time.replace("/", "_").replace(" ", "_").replace(":", "")
    # 抓出網址內的內文，先送requests.get，再去抓單篇文章的內容
    article_resp = requests.get(link, headers=head)
    soup = BeautifulSoup(article_resp.text, "html.parser")

    article_p_list = soup.find("div", {"id": "article_body"}).find_all("p")

    with codecs.open(f"1013_news/{time_mdf}-{title}.txt", "w", encoding="utf-8") as save_file:
        # 開啟檔案後寫入存在article_p_list裡面抓到的文章段落
        for article_p in article_p_list:
            save_file.write(article_p.text)
        # 在文末加入斷行與文章連結    
        save_file.write("\n")
        save_file.write(link)
    